In [3]:
import torch
import got_utils
import numpy as np
import pandas as pd
import os
import json

In [11]:
class CodingData:
    def __init__(self, ROOT):
        self.act = np.load(os.path.join(ROOT, 'act.npy')) # (Samples, Layers, Dim)
        # with open(os.path.join(ROOT, 'labeled_data.json')) as f:
        #     self.data = json.load(f)
        self.data = pd.read_json(os.path.join(ROOT, 'labeled_data.json'))

In [13]:
data = CodingData('data/datasets/factual/easy_100')

In [14]:
data.act.shape

(99, 33, 4096)

In [16]:
data.data.head()

,query,response,label
0,What is the name of a python package or librar...,[INST] <<SYS>>\nYou are a helpful AI assistant...,True
1,What is the name of a python package or librar...,[INST] <<SYS>>\nYou are a helpful AI assistant...,False
2,What is the name of a python package or librar...,[INST] <<SYS>>\nYou are a helpful AI assistant...,False
3,What is the name of a python package or librar...,[INST] <<SYS>>\nYou are a helpful AI assistant...,False
4,What is the name of a python package or librar...,[INST] <<SYS>>\nYou are a helpful AI assistant...,False
